In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt

In [59]:
repr_dic = "/shared/results/common/kargin/unreal_engine/features/initial_test"
MODEL = "spa"
result = torch.load(f"{repr_dic}/repr_{MODEL}.pt")

In [60]:
# --- Manual filtering ---
mask = [
    env == 'City' 
    and obj == 'SM_vehCar_vehicle07_White' 
    and ori == 'Line'
    for env, obj, ori in zip(result['environment'], result['object_class'], result['orientation'])
]

In [61]:
FEATURES = "CLS"

features = torch.stack([f for f, m in zip(result['features'], mask) if m]).to(torch.float32)
positions = torch.stack([a for a, m in zip(result['number'], mask) if m]).to(torch.float32) / 360.0

if FEATURES == "CLS":
    features = features[:, 0, :]
elif FEATURES == "MEAN":
    features = features[:, -196:, :].mean(1)
elif FEATURES == "CENTER":
    features = features[:, 105, :]
else:
    raise Exception("bruh")


In [ ]:
# sort the features by the angles
features = features[torch.argsort(positions)]
lines = positions[torch.argsort(positions)]


# --- Split ---
train_mask = lines < 0.9 # Time-based split
# train_mask = torch.rand(len(lines)) < 0.7 # Random split
val_mask = ~train_mask
train_x, train_y = features[train_mask], lines[train_mask]
val_x, val_y = features[val_mask], lines[val_mask]

# --- DataLoader ---
train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=64, shuffle=True)

# --- Model ---
model = nn.Sequential(
    nn.Linear(train_x.shape[1], 256),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(256, 1)
)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
loss_fn = nn.MSELoss()
train_loss_list = []
val_loss_list = []
# --- Training ---
for epoch in range(300):
    pbar = tqdm(train_loader)
    for x, y in pbar:
        pred = model(x).squeeze(1)# .sigmoid()
        loss = loss_fn(pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        pbar.set_description(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            preds = model(val_x).squeeze(1)# .sigmoid()
            val_loss = loss_fn(preds, val_y).item()
            val_loss_list.append(val_loss)
            train_loss_list.append(loss.item())
        model.train()


# --- Validation ---
model.eval()
with torch.no_grad():
    preds_val = model(val_x).squeeze(1)# .sigmoid()
    val_loss = loss_fn(preds_val, val_y).item()
    preds_train = model(train_x).squeeze(1)# .sigmoid()
    train_loss = loss_fn(preds_train, train_y).item()
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

In [ ]:
# plot loss
plt.plot(train_loss_list, label='Training Loss')
plt.plot(val_loss_list, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Detach tensors and convert to numpy for plotting
true_xy = train_y.numpy()
pred_xy = preds_train.numpy()

# Plot ground truth
plt.figure(figsize=(6, 6))
plt.scatter(range(len(true_xy)), true_xy, c='blue', label='Ground Truth', alpha=0.6)

# Plot predictions
plt.scatter(range(len(pred_xy)), pred_xy, c='red', label='Predictions', alpha=0.6)

# plt.axis('equal')
plt.ylim(-0.1, 1)
plt.title("Predicted vs. Ground Truth Positions on Line")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Detach tensors and convert to numpy for plotting
true_xy = val_y.numpy()
pred_xy = preds_val.numpy()

# Plot ground truth
plt.figure(figsize=(6, 6))
plt.scatter(range(len(true_xy)), true_xy, c='blue', label='Ground Truth', alpha=0.6)

# Plot predictions
plt.scatter(range(len(pred_xy)), pred_xy, c='red', label='Predictions', alpha=0.6)

# Draw connection lines
for i, (gt, pred) in enumerate(zip(true_xy, pred_xy)):
    plt.plot([i, i], [gt, pred], c='gray', alpha=0.7, linewidth=0.8)

# plt.axis('equal')
plt.ylim(0.5, 1.2)
plt.title("Predicted vs. Ground Truth Positions on Line")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)
plt.show()